In [16]:
# Loading data

import numpy as np
import pandas as pd
import json

with open("/home/data/exercise/normalised.json") as datafile:
    data = json.load(datafile)
exercises = pd.DataFrame(data)

# Normalised data loaded

exercises.columns = ['date', 'exercise', 'weight']

In [17]:
exercises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      681 non-null    object
 1   exercise  681 non-null    object
 2   weight    681 non-null    object
dtypes: object(3)
memory usage: 16.1+ KB


In [18]:
exercises['date'] = pd.to_datetime(exercises['date'])
exercises['exercise'] = exercises['exercise'].astype(str)
exercises['weight'] = exercises['weight'].astype(float)

In [19]:
exercises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      681 non-null    datetime64[ns]
 1   exercise  681 non-null    object        
 2   weight    681 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 16.1+ KB


In [20]:
# indices with blank data
missing = exercises.index[exercises.isnull().any(axis=1)]
# rows at those indices with missing data
exercises.loc[missing]

,date,exercise,weight


# Most popular exercises

In [21]:
n_by_exercise = exercises.groupby("exercise")["date"].count().sort_values(ascending=False)
print(n_by_exercise)

exercise
Converging Chest Press       98
Triceps Pull                 92
Converging Shoulder Press    81
Pectoral Fly                 76
Leg Extension                70
Seated Leg Curl              68
Lateral Raise                49
Leg press                    48
Arm Curl                     45
Lat Pull                     31
Dual Hand Pull               11
Diverging Seated Row          8
Mini Ripper                   2
Triceps Extension             2
Name: date, dtype: int64


# Increase in weight per exercise

In [22]:
# get max date
max_date = exercises.loc[exercises['date'].idxmax()]['date']

# get min date
min_date = exercises.loc[exercises['date'].idxmin()]['date']

# number of months difference
month_delta = (max_date.year - min_date.year) * 12 + (max_date.month - min_date.month)

In [23]:
weight_per_exercise = exercises.groupby("exercise")['weight'].max().sort_values(ascending=False)
weight_per_exercise
# add new derived column
# exercises['weight_per_month'] = exercises['weight'] / month_delta

exercise
Leg press                    86.0
Pectoral Fly                 86.0
Converging Chest Press       61.3
Seated Leg Curl              56.6
Leg Extension                54.3
Lat Pull                     52.0
Lateral Raise                50.0
Dual Hand Pull               45.0
Triceps Extension            45.0
Arm Curl                     37.1
Diverging Seated Row         36.0
Converging Shoulder Press    32.0
Mini Ripper                  32.0
Triceps Pull                 29.3
Name: weight, dtype: float64

# Rate of weight increase per exercise

# Number of times I've exercised

In [24]:
exercises.date.nunique()

145

# Average number of weight exercises per session

In [25]:
exercises.weight.size / exercises.date.nunique()

4.696551724137931

# Questions it would be good to answer

* increase in weight over time as raw number
* increase in weight over time as percentage of starting weight

In [26]:
grouped = exercises.groupby("exercise")
min_max_df = grouped.agg(min_weight=('weight', 'min'), max_weight=('weight', 'max'), min_date=('date', 'min'), max_date=('date', 'max')).reset_index()
min_max_df["delta"] = min_max_df["max_weight"] - min_max_df["min_weight"]

In [27]:
min_max_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   exercise    14 non-null     object        
 1   min_weight  14 non-null     float64       
 2   max_weight  14 non-null     float64       
 3   min_date    14 non-null     datetime64[ns]
 4   max_date    14 non-null     datetime64[ns]
 5   delta       14 non-null     float64       
dtypes: datetime64[ns](2), float64(3), object(1)
memory usage: 800.0+ bytes


In [28]:
# Calculate number of months since from first to last exercise
min_max_df["delta_months"] = (min_max_df["max_date"].dt.year - min_max_df["min_date"].dt.year) * 12 + (min_max_df["max_date"].dt.month - min_max_df["min_date"].dt.month)

# From weight change, calculate average change per month for exercise
min_max_df["delta_per_month"] = min_max_df["delta"] / min_max_df["delta_months"]

min_max_df.loc[:, ['exercise', 'delta_per_month']].sort_values(by=['delta_per_month'], ascending=False)

,exercise,delta_per_month
8,Leg press,0.755556
10,Pectoral Fly,0.577778
6,Lateral Raise,0.534884
7,Leg Extension,0.495556
1,Converging Chest Press,0.354348
11,Seated Leg Curl,0.332609
2,Converging Shoulder Press,0.311111
5,Lat Pull,0.222222
13,Triceps Pull,0.202174
12,Triceps Extension,0.121212
